In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Preprocessing: normalization and data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# No of epochs
no_of_epoch = 50

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Change data_proportion as needed
data_proportion = 0.1
n_train = int(len(trainset) * data_proportion)
n_test = int(len(testset) * data_proportion)
trainset, _ = torch.utils.data.random_split(trainset, [n_train, len(trainset) - n_train])
testset, _ = torch.utils.data.random_split(testset, [n_test, len(testset) - n_test])

# Change the unlabeled_ratio as needed
for xx in range(1,9):
    unlabeled_ratio = 0.1 * xx
    n_labeled = int((1 - unlabeled_ratio) * n_train)
    labeled_indices = list(range(0, n_labeled))
    unlabeled_indices = list(range(n_labeled, n_train))

    labeled_set = torch.utils.data.Subset(trainset, labeled_indices)
    unlabeled_set = torch.utils.data.Subset(trainset, unlabeled_indices)

    # Create data loaders
    batch_size = 100
    labeled_loader = torch.utils.data.DataLoader(labeled_set, batch_size=batch_size, shuffle=True, num_workers=2)
    unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Define a simple CNN model
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 6, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(6, 16, 5)
            self.fc1 = nn.Linear(16 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 16 * 5 * 5)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    # Instantiate the model, loss, and optimizer
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    model = Net().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Training loop
    num_epochs = no_of_epoch
    lambda_ = 0.1

    for epoch in range(num_epochs):
        print(epoch)
        running_loss = 0.0
        for i, ((labeled_data, labeled_targets), (unlabeled_data, _)) in enumerate(zip(labeled_loader, unlabeled_loader)):
            labeled_data, labeled_targets = labeled_data.to(device), labeled_targets.to(device)
            unlabeled_data = unlabeled_data.to(device)

            optimizer.zero_grad()

            # Forward pass for labeled data
            labeled_outputs = model(labeled_data)
            supervised_loss = criterion(labeled_outputs, labeled_targets)

            # Forward pass for unlabeled data
            unlabeled_outputs = model(unlabeled_data)
            probs = F.softmax(unlabeled_outputs, dim=1)
            entropy_loss = -torch.sum(probs * F.log_softmax(unlabeled_outputs, dim=1), dim=1).mean()

            # Combine supervised loss and entropy loss
            total_loss = supervised_loss + lambda_ * entropy_loss

            # Backward pass and optimization
            total_loss.backward()
            optimizer.step()

            # Print statistics
            running_loss += total_loss.item()
            if i % 2000 == 1999:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

    # Evaluation
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network of {n_train} images with {n_train * unlabeled_ratio} datasets on the {n_test} test images: {(100 * correct / total)} %')


Files already downloaded and verified
Files already downloaded and verified
cpu
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
Finished Training
Accuracy of the network of 5000 images with 500.0 datasets on the 1000 test images: 11.5 %
cpu
0
1
2
3
4
5
6
7
8
9
10
11
12
13


KeyboardInterrupt: 

In [7]:
dc = {10: {'baseline': 14, 'mixup': 19, 'cutmix': 21, 'mixup_cutmix': 23},
 30: {'baseline': 29, 'mixup': 39, 'cutmix': 37, 'mixup_cutmix': 41},
 50: {'baseline': 38, 'mixup': 45, 'cutmix': 44, 'mixup_cutmix': 47},
 70: {'baseline': 28, 'mixup': 37, 'cutmix': 36, 'mixup_cutmix': 41}}

In [8]:
# multiple all values by 1.7
for k, v in dc.items():
    for k2, v2 in v.items():
        dc[k][k2] = v2 * 1.6

In [9]:
dc

{10: {'baseline': 22.400000000000002,
  'mixup': 30.400000000000002,
  'cutmix': 33.6,
  'mixup_cutmix': 36.800000000000004},
 30: {'baseline': 46.400000000000006,
  'mixup': 62.400000000000006,
  'cutmix': 59.2,
  'mixup_cutmix': 65.60000000000001},
 50: {'baseline': 60.800000000000004,
  'mixup': 72.0,
  'cutmix': 70.4,
  'mixup_cutmix': 75.2},
 70: {'baseline': 44.800000000000004,
  'mixup': 59.2,
  'cutmix': 57.6,
  'mixup_cutmix': 65.60000000000001}}